# Executing Shell Commands using Python [origin](https://gist.github.com/nikhilkumarsingh/c23fbd592b4268f225cb895afaa187fe?short_path=45867bd)

![](https://i.imgur.com/hV6TPQX.png)

## 1. Older methods


- [os.system](https://docs.python.org/3/library/os.html#os.system)
    - Execute the command (a string) in a subshell.
    - On Unix, the return value is the exit status of the process.
    - On Windows, the return value is that returned by the system shell after running command.
- [os.popen](https://docs.python.org/3/library/os.html#os.popen)
    - Open a pipe to or from command being executed.
    - The return value is an open file object connected to the pipe, which can be read or written depending on whether mode is 'r' (default) or 'w'.
    - The `close` method returns None if the subprocess exited successfully, or the subprocess’s return code if there was an error.
- [os.spawn*](https://docs.python.org/3/library/os.html#os.spawnl)
    - Execute the program *path* in a new process.
    - If *mode* is `P_NOWAIT`, this function returns the process id of the new process.
    - If *mode* is `P_WAIT`, returns the process’s exit code if it exits normally, or -signal, where signal is the signal that killed the process.


In [1]:
import os

In [11]:
!ls

Debug-Python-With-VS-Code.md shell.ipynb


In [12]:
os.system("ls")

0

In [15]:
os.system("rm xyz")

256

In [16]:
print(os.popen("ls").read())

Debug-Python-With-VS-Code.md
shell.ipynb



In [6]:
os.spawnl(os.P_NOWAIT, "/usr/sh", "ls")

47731

## 2. [`subprocess`](https://docs.python.org/3/library/subprocess.html) module

Provides more powerful ways to manage and communicate with subprocesses.

### Main API
---
- [`run(...)`](https://docs.python.org/3/library/subprocess.html#subprocess.run)
<br>
    Runs a command, waits for it to complete, then returns a `CompletedProcess` instance. (added in Python 3.5)


- [`Popen(...)`](https://docs.python.org/3/library/subprocess.html#popen-constructor)<br>
    A class for flexibly executing a command in a new process


### [Older API (prior to Python 3.5)](https://docs.python.org/3/library/subprocess.html#older-high-level-api)
----
- `call(...)` <br>
    Runs a command, waits for it to complete, then returns the return code.


- `check_call(...)` <br>
    Same as `call()` but raises `CalledProcessError()` if return code is not 0


- `check_output(...)`<br>
    Same as `check_call()` but returns the contents of stdout instead of a return code


- `getoutput(...)`<br>
    Runs a command in the shell, waits for it to complete, then returns the output


- `getstatusoutput(...)`<br>
    Runs a command in the shell, waits for it to complete, then returns a (exitcode, output) tuple
    
-----

### Exploring `subprocess.run`

### 1. run command

In [7]:
import subprocess

In [8]:
subprocess.run("ls")

CompletedProcess(args='ls', returncode=0)

In [9]:
subprocess.run("python3 test.py")

FileNotFoundError: [Errno 2] No such file or directory: 'python3 test.py'

In [ ]:
subprocess.run(["python3", "test.py"])

CompletedProcess(args=['python3', 'test.py'], returncode=2)

### 2. run command and read output

#### Prior to Python 3.7

`subprocess.PIPE` opens a pipe to the standard stream.

- `stdout=subprocess.PIPE`
- `stderr=subprocess.PIPE`
- `stderr=subprocess.STDOUT`

In [ ]:
result = subprocess.run("ls", stdout=subprocess.PIPE)

In [ ]:
print(result.stdout.decode())

1.flv
2.flv
3.flv
4.flv
5.flv
Tutorial.ipynb



In [ ]:
subprocess.run(["rm", "xyz"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

CompletedProcess(args=['rm', 'xyz'], returncode=1, stdout=b'', stderr=b"rm: cannot remove 'xyz': No such file or directory\n")

In [ ]:
subprocess.run(["rm", "xyz"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

CompletedProcess(args=['rm', 'xyz'], returncode=1, stdout=b"rm: cannot remove 'xyz': No such file or directory\n")

#### Since Python 3.7

If `capture_output` is true, `stdout` and `stderr` will be automatically captured. 

`stdout=PIPE` and `stderr=PIPE` is set automatically

In [ ]:
subprocess.run("ls", capture_output=True)

CompletedProcess(args='ls', returncode=0, stdout=b'1.flv\n2.flv\n3.flv\n4.flv\n5.flv\nTutorial.ipynb\n', stderr=b'')

### 3. run command by passing single string

>`args` is required for all calls and should be a string, or a sequence of program arguments. Providing a sequence of arguments is generally preferred, as it allows the module to take care of any required escaping and quoting of arguments (e.g. to permit spaces in file names). 
>
>If passing a single string, either `shell` must be `True` or else the string must simply name the program to be executed without specifying any arguments.



In [ ]:
subprocess.run("ls -a", shell=True)

CompletedProcess(args='ls -a', returncode=0)

### [Security Considerations](https://docs.python.org/3/library/subprocess.html#security-considerations) before using `shell=True`.

>Unlike some other popen functions, this implementation will never implicitly call a system shell. This means that all characters, including shell metacharacters, can safely be passed to child processes. If the shell is invoked explicitly, via shell=True, it is the application’s responsibility to ensure that all whitespace and metacharacters are quoted appropriately to avoid **shell injection** vulnerabilities.

When using `shell=True`, the `shlex.quote()` function can be used to properly escape whitespace and shell metacharacters in strings that are going to be used to construct shell commands.

In [ ]:
user_input = "a.txt ; pwd"
command = "cat {}".format(user_input)

In [ ]:
subprocess.run(command, shell=True, capture_output=True)

CompletedProcess(args='cat a.txt ; pwd', returncode=0, stdout=b'/home/nikhil/Videos/shell\n', stderr=b'cat: a.txt: No such file or directory\n')

In [ ]:
import shlex

In [ ]:
command = "cat {}".format(shlex.quote(user_input))

In [ ]:
command

"cat 'a.txt ; pwd'"

In [ ]:
subprocess.run(command, shell=True, capture_output=True)

CompletedProcess(args="cat 'a.txt ; pwd'", returncode=1, stdout=b'', stderr=b"cat: 'a.txt ; pwd': No such file or directory\n")

In [ ]:
shlex.split("cat 'abc def' >> a.txt")

['cat', 'abc def', '>>', 'a.txt']

### 4. run command and pass input

- `input` param

- `stdin` param

In [ ]:
!cat test.py

cat: test.py: No such file or directory


In [ ]:
subprocess.run(["python3", "test.py"], capture_output=True, input="abc\ndef".encode())

CompletedProcess(args=['python3', 'test.py'], returncode=2, stdout=b'', stderr=b"python3: can't open file 'test.py': [Errno 2] No such file or directory\n")

In [ ]:
subprocess.run(["python3", "test.py"], capture_output=True, input="abc\ndef", 
               universal_newlines=True)

CompletedProcess(args=['python3', 'test.py'], returncode=2, stdout='', stderr="python3: can't open file 'test.py': [Errno 2] No such file or directory\n")

In [ ]:
subprocess.run(["python3", "test.py"], capture_output=True, input="abc\ndef", text=True)

CompletedProcess(args=['python3', 'test.py'], returncode=2, stdout='', stderr="python3: can't open file 'test.py': [Errno 2] No such file or directory\n")

In [ ]:
!cat a.txt

cat: a.txt: No such file or directory


In [ ]:
subprocess.run(["python3", "test.py"], capture_output=True, stdin=open("a.txt", 'r'))

FileNotFoundError: [Errno 2] No such file or directory: 'a.txt'

### 5. run command with timeout

Set `timeout` parameter.

In [ ]:
subprocess.run(["sleep", "5"], timeout=3)

TimeoutExpired: Command '['sleep', '5']' timed out after 2.9998896930001138 seconds

In [ ]:
subprocess.run(["sleep", "3"], timeout=5)

CompletedProcess(args=['sleep', '3'], returncode=0)

### 6. run command and throw error if fail

In [ ]:
subprocess.run(["rm", "xyz"])

CompletedProcess(args=['rm', 'xyz'], returncode=1)

In [ ]:
try:
    subprocess.run(["rm", "xyz"], check=True)
except subprocess.CalledProcessError:
    print("failed")

failed
